In [30]:
from gurobipy import Model, GurobiError, GRB

In [5]:
# Make sure that Gurobi works and that you
# have a valid license.

try:
    Model()
except GurobiError as e:
    print(e.message)

Exercise from the lecture notes.

In [5]:
from enum import Enum

In [6]:
class Petrol(Enum):
    A = 'RawA'
    B = 'RawB'
    C = 'RawC'
    D = 'RawD'

class Fuel(Enum):
    A = 'FuelA'
    B = 'FuelB'

pn = {
    Petrol.A: 107,
    Petrol.B: 93,
    Petrol.C: 87,
    Petrol.D: 108
}

rvp = {
    Petrol.A: 5,
    Petrol.B: 8,
    Petrol.C: 4,
    Petrol.D: 21
}

prod = {
    Petrol.A: 3814,
    Petrol.B: 2666,
    Petrol.C: 4016,
    Petrol.D: 1300
}

petrol_profit = 4.83

fuel_profit = {
    Fuel.A: 6.45,
    Fuel.B: 5.91
}

min_pn = {
    Fuel.A: 100,
    Fuel.B: 91
}

max_rvp = {
    Fuel.A: 7,
    Fuel.B: 7
}

In [9]:
m = Model()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2555908
Academic license 2555908 - for non-commercial use only - registered to al___@upf.edu


In [25]:
x = m.addVars(Petrol, Fuel, name='x')

In [31]:
m.setObjective(
    sum(
        fuel_profit[j] *
        sum(
            x[i,j]
            for i in Petrol)
        for j in Fuel) +
    petrol_profit *
    sum(
        prod[i] -
        sum(
            x[i,j]
            for j in Fuel)
        for i in Petrol),
sense=GRB.MAXIMIZE)

In [ ]:
# m.addConstrs(
#     (sum(x[i,j] for j in Fuel) <= prod[i]
#     for i in Petrol),
# name='respect_production_capacity')

In [33]:
m.addConstrs(
    (x.sum(i,'*') <= prod[i]
    for i in Petrol),
name='respect_production_capacity')

{<Petrol.A: 'RawA'>: <gurobi.Constr *Awaiting Model Update*>,
 <Petrol.B: 'RawB'>: <gurobi.Constr *Awaiting Model Update*>,
 <Petrol.C: 'RawC'>: <gurobi.Constr *Awaiting Model Update*>,
 <Petrol.D: 'RawD'>: <gurobi.Constr *Awaiting Model Update*>}

In [ ]:
# m.addConstrs(
#     (sum(pn[i] * x[i,j] for i in Petrol) >=
#      min_pn[j] * sum(x[i,j] for i in Petrol)
#     for j in Fuel),
# name='min_pn_requirements')

In [35]:
m.addConstrs(
    (sum(pn[i] * x[i,j] for i in Petrol) >=
     min_pn[j] * x.sum('*',j)
    for j in Fuel),
name='min_pn_requirements')

{<Fuel.A: 'FuelA'>: <gurobi.Constr *Awaiting Model Update*>,
 <Fuel.B: 'FuelB'>: <gurobi.Constr *Awaiting Model Update*>}

In [36]:
m.addConstrs(
    (sum(rvp[i] * x[i,j] for i in Petrol) <=
     max_rvp[j] * x.sum('*',j)
    for j in Fuel),
name='max_rvp_requirements')

{<Fuel.A: 'FuelA'>: <gurobi.Constr *Awaiting Model Update*>,
 <Fuel.B: 'FuelB'>: <gurobi.Constr *Awaiting Model Update*>}

In [37]:
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-8550U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2555908 - for non-commercial use only - registered to al___@upf.edu
Optimize a model with 8 rows, 16 columns and 24 nonzeros
Model fingerprint: 0x7716a408
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 4e+03]
Presolve removed 1 rows and 9 columns
Presolve time: 0.02s
Presolved: 7 rows, 7 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.6084200e+04   1.743529e+03   0.000000e+00      0s
       5    7.3879380e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.03 seconds (0.00 work units)
Optimal objective  7.387938000e+04


In [41]:
for j in Fuel:
    tot = sum(x[i,j].X for i in Petrol)

    print(f"=== Fuel {j.name} ===")
    print(f"Total produced fuel: {tot:.1f}")
    print(f"Total revenue from this fuel: {tot*fuel_profit[j]:.1f}")
    print("Composition:")
    for i in Petrol:
        if (val := x[i,j].X) > 0:
            print(f"\t Using {val:.1f} barrels of petrol {i.name}")

=== Fuel A ===
Total produced fuel: 7883.0
Total revenue from this fuel: 50845.3
Composition:
	 Using 3754.0 barrels of petrol A
	 Using 2666.0 barrels of petrol B
	 Using 920.0 barrels of petrol C
	 Using 543.0 barrels of petrol D
=== Fuel B ===
Total produced fuel: 3828.0
Total revenue from this fuel: 22623.5
Composition:
	 Using 60.0 barrels of petrol A
	 Using 3096.0 barrels of petrol C
	 Using 672.0 barrels of petrol D
